In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import trackpy as tp
import math
from scipy.misc import imsave
from scipy.misc import imread
import re
################################################
# Read frames which have been data-cleaned (wire_particle_tracking_datacleaning). 
################################################
shot = 232
picL = 18333
picR = 18158
BR = [[5.7149E-08,1.1882E-05,-2.4321E-03],[8.8831E-06,-5.3566E-07,-3.8744E-02],[-1.6469E-03,3.3774E-02,9.9867E-01]]
TR = [[-5.7704E-08,1.1998E-05,-2.1514E-03],[8.9694E-06,5.4087E-07,-3.9328E-02],[-1.7813E-03,3.3894E-02,9.9865E-01]]
TL = [[1.1040E-07,2.2954E-05,-4.1584E-03],[1.7160E-05,-1.0348E-06,6.8654E-02],[-3.4505E-03,-7.3661E-02,9.9490E-01]]
BL = [[-1.0175E-07,2.1155E-05,-4.2910E-03],[1.5815E-05,9.5369E-07,6.2907E-02],[-2.8931E-03,-6.8254E-02,9.9567E-01]]
F_10point = [[  8.34360564e-08,   1.89022714e-05,  -3.73041652e-03],[  1.94501963e-05,   1.19512678e-06,  -5.90813947e-02],[ -3.16737708e-03,   5.30589622e-02,   9.96830069e-01]]
#shot115,235
F_10point_2 = [[  5.59273094e-07,  -6.08650734e-06,   2.05586754e-03], [  7.17519639e-06,   1.44074201e-06,  -7.14172969e-03],[ -3.18532599e-03,   4.59309570e-03,   9.99956762e-01]]
F_big = [[  1.75345969e-07,   1.87353876e-05,  -3.69560210e-03], [  1.92508173e-05,   9.79548859e-07,  -5.72239187e-02],[ -3.15390413e-03,   5.12220927e-02,   9.97034661e-01]]
F = F_big
Matr = F
MatrixL = np.array(Matr)
MatrixR = MatrixL.transpose()
Linex = np.arange(0,384,1)
v0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL_sum.tif'.format(shot,shot,picL))
v0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR_sum.tif'.format(shot,shot,picR))
bk0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL0.tif'.format(shot,shot,picL))
bk0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR0.tif'.format(shot,shot,picR))

In [2]:
tL = pd.read_csv('./Data/Shot{}/trackL3_frame_inv.csv'.format(shot))
tL.head()
#tL['particle'].unique()

,frame,x,y,mass,size,ecc,signal,raw_mass,ep,frame.1,particle,range
0,0,207.004950,81.023515,144.511368,0.836216,0.428319,43.460721,359.0,0.000165,0,0.0,137.171333
1,1,207.009604,81.025210,147.688042,0.836588,0.423671,44.324142,367.0,0.000340,1,0.0,137.171333
2,2,207.016355,81.032710,153.277122,0.838613,0.418605,45.481763,371.0,0.000300,2,0.0,137.171333
3,3,207.008383,81.028743,148.846725,0.837018,0.422222,44.564888,369.0,0.000292,3,0.0,137.171333
4,4,207.019394,81.050909,146.198923,0.837022,0.430796,43.771071,353.0,0.000237,4,0.0,137.171333


In [3]:
tR = pd.read_csv('./Data/Shot{}/trackR3_frame_inv.csv'.format(shot))
tR.head()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,0,NaN,NaN,NaN,NaN,NaN,0.383013,0.000317,0,184.147963,0.0,457.0,53.421764,0.842554,141.918089,78.879408,131.223595
1,0,NaN,NaN,NaN,NaN,NaN,0.346442,0.000418,0,150.628425,17.0,347.0,38.756966,0.861799,86.942976,282.948540,154.310801
2,1,NaN,3.0,NaN,NaN,NaN,0.384491,0.000321,1,179.520687,0.0,448.0,52.377317,0.841569,141.921053,78.881007,131.223595
3,1,NaN,3.0,NaN,NaN,NaN,0.344954,0.000413,1,150.764513,17.0,348.0,38.820835,0.861688,86.937330,282.946866,154.310801
4,2,NaN,3.0,NaN,NaN,NaN,0.385113,0.000223,2,178.049238,0.0,446.0,51.862966,0.841852,141.920872,78.881881,131.223595


In [4]:
## Print out particle track list in the left camera
listL = tL['particle'].unique()
listL = listL.astype(int)
listR = tR['particle'].unique()
listR = listR.astype(int)

In [5]:
for ilistL in listL:
    ipL0 = int(ilistL)
    TrackL = tL[tL['particle']==ipL0]
    lenL = len(TrackL)
    if ipL0>=0 and lenL>400:
        print(ipL0,lenL)    
        plt.figure(figsize = (24,24))  # make a new figure
        f, (ax1, ax2) = plt.subplots(1, 2)
        ax1.imshow(v0L+bk0L)
        ax1.scatter(TrackL['x'],TrackL['y'],color='r',marker=".",s=1)
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax2.scatter(TrackL['frame'],TrackL['mass'],color='r',marker=".",s=1)
        ax2.set_xlabel('Frame')
        ax2.set_ylabel('Mass')
        plt.savefig('./Data/Shot{}/TrackL_{}.pdf'.format(shot,ipL0))

(0, 6216)
(1302, 409)
(1855, 5162)
(2343, 4719)
(2388, 3836)
(2941, 2209)
(2943, 4368)
(2988, 3361)
(2998, 3664)
(3010, 2705)
(3106, 3928)


/Users/pinghanchu/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


(3467, 4055)
(3586, 3855)
(3705, 3754)
(3760, 4001)
(3871, 2393)
(3886, 3018)
(4341, 3517)
(4613, 2911)
(4692, 1813)
(4784, 2686)
(4820, 3256)
(4893, 812)
(4899, 2916)
(4913, 3267)
(4914, 3114)
(4918, 3090)
(4919, 1823)
(4926, 1595)
(4928, 2918)
(4929, 1808)
(4940, 527)
(4946, 2725)
(4964, 752)
(4972, 1962)
(4979, 2077)
(4984, 653)
(4985, 518)
(4987, 2215)
(4991, 1844)
(5011, 2667)
(5019, 2799)
(5024, 1393)
(5028, 2616)
(5029, 1336)
(5031, 1680)
(5032, 1499)
(5035, 1550)
(5036, 2273)
(5053, 1506)
(5058, 2419)
(5069, 2298)
(5074, 2193)
(5087, 2259)
(5114, 944)
(5126, 883)
(5138, 1848)
(5143, 591)
(5150, 2012)
(5214, 1423)
(5378, 1334)
(5395, 1316)
(5409, 2354)
(5418, 2129)
(5422, 683)
(5427, 1179)
(5432, 790)
(5448, 968)
(5457, 2106)
(5464, 418)
(5471, 1021)
(5493, 1342)
(5494, 1201)
(5495, 424)
(5510, 1079)
(5511, 658)
(5519, 2046)
(5521, 1768)
(5540, 666)
(5548, 1961)
(5562, 1088)
(5563, 1627)
(5569, 1345)
(5570, 1858)
(5576, 1713)
(5577, 640)
(5579, 1578)
(5607, 1492)
(5623, 1517)
(5

In [6]:
for ilistR in listR:
    ipR0 = int(ilistR)
    TrackR = tR[tR['particle']==ipR0]
    lenR = len(TrackR)
    if ipR0>=0:
        print(ipR0,lenR)    
        plt.figure(figsize = (24,24))  # make a new figure
        f, (ax1, ax2) = plt.subplots(1, 2)
        ax1.imshow(v0R+bk0R)
        ax1.scatter(TrackR['x'],TrackR['y'],color='r',marker=".",s=1)
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax2.scatter(TrackR['frame'],TrackR['mass'],color='r',marker=".",s=1)
        ax2.set_xlabel('Frame')
        ax2.set_ylabel('Mass')
        plt.savefig('./Data/Shot{}/TrackR_{}.pdf'.format(shot,ipR0))

(0, 6521)
(17, 6246)
(277, 784)
(408, 415)
(916, 5889)
(1541, 3712)
(1561, 3620)
(1773, 3585)
(1915, 4214)
(1937, 3159)
(1968, 4269)
(2135, 3300)
(2287, 4399)
(2573, 2810)
(2587, 4028)
(2673, 3663)
(2687, 3492)
(2757, 3181)
(2778, 3432)
(2920, 1464)
(3005, 1804)
(3061, 3597)
(3080, 3761)
(3242, 3522)
(3260, 3286)
(3265, 3243)
(3280, 3406)
(3290, 2493)
(3296, 3152)
(3308, 1136)
(3312, 2914)
(3313, 3231)
(3322, 2899)
(3335, 611)
(3337, 1286)
(3356, 1050)
(3369, 3038)
(3378, 2831)
(3382, 2614)
(3384, 2832)
(3385, 1813)
(3386, 1480)
(3387, 1977)
(3389, 2569)
(3393, 2728)
(3395, 2543)
(3396, 2600)
(3398, 2134)
(3402, 575)
(3406, 2376)
(3421, 2093)
(3424, 2653)
(3451, 2522)
(3481, 1014)
(3497, 1355)
(3507, 910)
(3509, 861)
(3523, 479)
(3532, 251)
(3546, 1906)
(3553, 1552)
(3554, 1151)
(3572, 1365)
(3587, 1979)
(3591, 1602)
(3597, 2258)
(3613, 1377)
(3622, 1589)
(3630, 2056)
(3640, 2444)
(3645, 1164)
(3648, 860)
(3692, 2058)
(3737, 1775)
(3758, 2206)
(3773, 2185)
(3775, 1435)
(3783, 493)
(378

(5207, 375)
(5209, 233)
(5210, 426)
(5212, 361)
(5213, 265)
(5214, 394)
(5216, 321)
(5218, 214)
(5220, 421)
(5226, 219)
(5228, 232)
(5230, 320)
(5233, 325)
(5236, 329)
(5239, 435)
(5241, 365)
(5243, 373)
(5245, 209)
(5251, 411)
(5264, 284)
(5267, 319)
(5269, 314)
(5270, 242)
(5276, 300)
(5278, 272)
(5279, 369)
(5286, 316)
(5288, 213)
(5290, 269)
(5296, 416)
(5306, 409)
(5309, 235)
(5310, 343)
(5311, 306)
(5313, 389)
(5314, 382)
(5319, 275)
(5325, 396)
(5326, 328)
(5327, 298)
(5331, 372)
(5333, 380)
(5336, 409)
(5342, 253)
(5343, 380)
(5350, 375)
(5351, 331)
(5354, 278)
(5373, 346)
(5378, 362)
(5379, 310)
(5380, 331)
(5385, 310)
(5387, 289)
(5389, 339)
(5395, 283)
(5396, 284)
(5399, 330)
(5400, 240)
(5402, 233)
(5408, 343)
(5414, 320)
(5419, 385)
(5426, 247)
(5427, 378)
(5428, 225)
(5429, 352)
(5443, 286)
(5445, 229)
(5446, 271)
(5452, 335)
(5453, 307)
(5455, 216)
(5460, 241)
(5467, 361)
(5468, 215)
(5473, 377)
(5485, 266)
(5488, 244)
(5491, 396)
(5497, 203)
(5500, 210)
(5503, 235)
(550